<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 30px; margin: 0;">data-prep-kit planning agent</h1>   
</div>

In [ ]:
%pip install -qq -r requirements.txt

In [ ]:
from IPython.display import HTML
task = "Process the provided PDF dataset to identify and extract only documents that don't contain inappropriate language. Remove the duplications."
HTML(f"<p><span style='color:blue; font-weight:bold; font-size:14.0pt;'>TASK: {task}</span></p>")

In [ ]:
import logging
import os

from llm_utils.logging import prep_loggers
os.environ["LLM_LOG_PATH"] = "./logs/llm_log.txt"
prep_loggers("llm=INFO")

In [ ]:
# The tools in DPK agents are the transforms.
# Each tool is described as json dictionary with its name, description, input parameters, and how to import it.
# The list of the tools exists in llm_utils/tools.py file.
from llm_utils.dpk.tools import *
print(tools_json)

In [ ]:
# This is an example of a plan for a simple task. It is possed to the prompt to enhance the planning results.
from llm_utils.dpk.examples import *
print(example_task)

In [ ]:
# This is a string that contains several constraints on the order of the tools in the plan.
# It is a free text and can be found in llm_utils/constraints.py file.
from llm_utils.dpk.constraints import *
print(constraints)

## Define LLM models

We have have tested our project with the following LLM execution frameworks: [Watsonx](https://www.ibm.com/watsonx), [Replicate](https://replicate.com/), and locally running [Ollama](https://ollama.com/).
To use one of the frameworks uncomment its part in the cell below while commenting out the other frameworks.
Please note that the notebooks have been tested with specific Large Language Models (LLMs) that are mentioned in the cell, and due to the inherent nature of LLMs, using a different model may not produce the same results.

- To use Replicate:
  - Obtain Replicate API token
  - Store the following value in the `.env` file located in your project directory:
    ```
        REPLICATE_API_TOKEN=<your Replicate API token>
    ```
- To use Ollama: 
  - Download [Ollama](https://ollama.com/download).
  - Download one of the supported [models](https://ollama.com/search). We tested with `llama3.3` model.
  - update the `model_ollama_*` names if needed.
- To use Watsonx:
  - Register for Watsonx
  - Obtain its API key
  - Store the following values in the `.env` file located in your project directory:
    ```
        WATSONX_URL=<WatsonX entry point, e.g. https://us-south.ml.cloud.ibm.com>
        WATSON_PROJECT_ID=<your Watsonx project ID>
        WATSONX_APIKEY=<your Watsonx API key>
    ```

In [ ]:
from llm_utils.models import getChatLLM
from dotenv import dotenv_values

# watsonx part 
# config = dotenv_values("./.env")
# model_watsonx_id1 = "ibm-granite/granite-3.1-8b-instruct"
# model_watsonx_id2 = "meta-llama/llama-3-1-70b-instruct"
# model_watsonx_id3 = "meta-llama/llama-3-3-70b-instruct"
# model_watsonx_id4 = "ibm/granite-34b-code-instruct"

# llm_plan = getChatLLM("watsonx", model_watsonx_id2, config)
# llm_judge = getChatLLM("watsonx", model_watsonx_id2, config)
# llm_generate = getChatLLM("watsonx", model_watsonx_id2, config)

# # ollama part
# model_ollama = \"llama3.3\"\n",
# llm_plan = getChatLLM(\"ollama\", model_ollama);\n",
# llm_judge = getChatLLM(\"ollama\", model_ollama)\n",
# llm_generate = getChatLLM(\"ollama\", model_ollama)"

# replicate part
config = dotenv_values("./.env")
# You can use different llm models
model_replicate_id1 = "meta/meta-llama-3-70b-instruct"
llm_plan = getChatLLM("replicate", model_replicate_id1, config)
llm_judge = getChatLLM("replicate", model_replicate_id1, config)
llm_generate = getChatLLM("replicate", model_replicate_id1, config)

In [ ]:
from langgraph.graph import StateGraph, END
from llm_utils.agent_helpers import *
from llm_utils.prompts.planner_prompt import *
from llm_utils.prompts.judge_prompt import *
from llm_utils.prompts.generate_prompt import *
from llm_utils.dpk.tools import *
from llm_utils.dpk.examples import *
from llm_utils.dpk.constraints import *
from functools import partial


# Create the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("planner", partial(planner, prompt=planner_prompt_str, tools=tools_json, example=example_task1, context=constraints, llm=llm_plan))
workflow.add_node("judge", partial(judge, prompt=judge_prompt_str_dpk, tools=tools_json, context=constraints, llm=llm_judge))
workflow.add_node("user_review", get_user_review)
workflow.add_node("code generator", partial(generator, prompt=generate_prompt_str_with_example, llm=llm_generate))
workflow.add_node("code validator", code_validator_noop)

# Add edges
workflow.set_entry_point("planner")
workflow.add_edge("code generator", "code validator")
workflow.add_edge("code validator", END)

# Add conditional edges from judge
workflow.add_conditional_edges(
    "judge",
    is_plan_OK,
    {
        False: "planner",  # If needs revision, go back to planner
        True: "user_review"  # If plan is good, proceed to user review
    }
)

# Add conditional edges from planner
workflow.add_conditional_edges(
    "planner",
    need_judge,
    {
        True: "judge",  # If needs revision, go back to planner
        False: "user_review"  # If plan is good, proceed to user review
    }
)

workflow.add_conditional_edges(
    "user_review",
    is_user_review_OK,
    {
        False: "planner",  # If needs revision, go back to planner
        True: "code generator",
    }
)


In [ ]:
app = workflow.compile()

from IPython.display import Image, display

#display(Image(app.get_graph(xray=True).draw_mermaid_png()))
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# Run the graph
initial_state = {
    "task": task,
    "context": "",
    "plan": ["still no plan"],
    "planning_attempts": 0,
    "feedback": "Still no review",
    "needs_revision": "",
    "need_judge": True,
}

state = initial_state

for output in app.stream(state):
    pass